In [2]:
import torch
import torchvision
import torchvision.transforms as transforms
import torch.optim as optim
import time
import torch.nn.functional as F
import torch.nn as nn
from torchvision import models
import pandas as pd
import numpy as np
import os
try:
  import pytorch_lightning as pl
except:
  !pip install pytorch_lightning

In [3]:
device = torch.device("cuda:1" if torch.cuda.is_available() else "cpu")
print(device)

cuda:1


In [4]:
import sys
from google.colab import drive
drive.mount('/content/drive')
data = pd.read_csv(r'/content/drive/My Drive/AML/cassava-leaf-disease-classification/train.csv')
data.head()

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


,image_id,label
0,1000015157.jpg,0
1,1000201771.jpg,3
2,100042118.jpg,1
3,1000723321.jpg,1
4,1000812911.jpg,3


In [5]:
from torch.utils.data import Dataset, DataLoader
from PIL import Image

class CassavaDataset(Dataset):
    """ Cassava Dataset """
    
    def __init__(self, root_dir, transform=None, stage=None):
        if (stage):
            # We're in test stage then
            csv_output = pd.read_csv(os.path.join(root_dir, "sample_submission.csv"))
            self.images_dir = os.path.join(root_dir, "test_images")
        else:
            csv_output = pd.read_csv(os.path.join(root_dir, "train.csv"))
            self.images_dir = os.path.join(root_dir, "train_images")
        self.image_urls = np.asarray(csv_output["image_id"])
        self.labels = np.asarray(csv_output["label"])
        self.transform = transform
        
    def __len__(self):
        return len(self.image_urls)
    
    def __getitem__(self, idx):
        # Get and load image
        image_path = os.path.join(self.images_dir, self.image_urls[idx])
        image = Image.open(image_path)
        # Perform transforms if any
        if self.transform:
            image = self.transform(image)
        # Get label
        label = self.labels[idx]
        return image, label

In [6]:
from torch.utils.data import random_split
import math

class CassavaDataModule(pl.LightningDataModule):
    """ Cassava DataModule for Lightning """
    def __init__(self, root_dir, transform=None, batch_size=32):
        super().__init__()
        self.batch_size = batch_size
        self.root_dir = root_dir
        self.transform = transform
        
    def setup(self, stage=None):
        cassava_full = CassavaDataset(self.root_dir, self.transform)
        train_data_len = math.floor(len(cassava_full) * 0.7)
        val_data_len = len(cassava_full) - train_data_len
        # Create train and validation datasets
        self.cassava_train, self.cassava_val = random_split(cassava_full, [train_data_len, val_data_len], generator=torch.Generator().manual_seed(42))
        
        # Create test dataset
        self.cassava_test = CassavaDataset(self.root_dir, self.transform, stage="test")
        
    def train_dataloader(self):
        return DataLoader(self.cassava_train, batch_size=self.batch_size)
    
    def val_dataloader(self):
        return DataLoader(self.cassava_val, batch_size=self.batch_size)
    
    def test_dataloader(self):
        return DataLoader(self.cassava_test, batch_size=self.batch_size)

In [7]:
transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5)) # Standard Normalization
#      transforms.Normalize((0.485, 0.456, 0.406), (0.229, 0.224, 0.225)) #ImageNet Normalization
    ])

root_dir = r'/content/drive/My Drive/AML/cassava-leaf-disease-classification/'
cassava_data = CassavaDataModule(root_dir, transform, batch_size=64)
cassava_data.setup()

train_loader = cassava_data.train_dataloader()
val_loader = cassava_data.train_dataloader()
test_loader = cassava_data.train_dataloader()

In [8]:
import torchvision.models as models

class ImageNetModel(pl.LightningModule):
    def __init__(self, learning_rate=1e-3):
        super().__init__()
        
        # Set our learning rate
        self.learning_rate = learning_rate
        
        num_target_classes = 5
        self.feature_extractor = models.resnet50(pretrained=True)
        self.feature_extractor.eval()
        
        # Use the pretrained model to classify cassava
        self.classifier = nn.Linear(1000, num_target_classes)
        
        # Create loss function
        self.loss_fn = torch.nn.CrossEntropyLoss()
    
    def configure_optimizers(self):
        optimizer = torch.optim.Adam(self.parameters(), lr=self.learning_rate)
        return optimizer
    
    def forward(self, input_data):
        representations = self.feature_extractor(input_data)
        preds = self.classifier(representations)
        return preds
    
    def training_step(self, train_batch, batch_idx):
        x, y = train_batch
        predictions = self.forward(x)
        loss = self.loss_fn(predictions, y)
        return loss
    
    def validation_step(self, val_batch, batch_idx):
        x, y = val_batch
        predictions = self.forward(x)
        loss = self.loss_fn(predictions, y)
        self.log('val_loss', loss)

In [10]:
model = ImageNetModel()

trainer = pl.Trainer(gpus=1, auto_lr_find=True)

trainer.tune(model, cassava_data)

trainer.fit(model, cassava_data)

GPU available: True, used: True
TPU available: False, using: 0 TPU cores
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]

  | Name              | Type             | Params
-------------------------------------------------------
0 | feature_extractor | ResNet           | 25.6 M
1 | classifier        | Linear           | 5.0 K 
2 | loss_fn           | CrossEntropyLoss | 0     
-------------------------------------------------------
25.6 M    Trainable params
0         Non-trainable params
25.6 M    Total params
102.248   Total estimated model params size (MB)
/usr/local/lib/python3.7/dist-packages/pytorch_lightning/utilities/distributed.py:68: UserWarning: The dataloader, val dataloader 0, does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` (try 4 which is the number of cpus on this machine) in the `DataLoader` init to improve performance.
  warnings.warn(*args, **kwargs)
/usr/local/lib/python3.7/dist-packages/pytorch_lightning/uti

/usr/local/lib/python3.7/dist-packages/pytorch_lightning/utilities/distributed.py:68: UserWarning: Detected KeyboardInterrupt, attempting graceful shutdown...
  warnings.warn(*args, **kwargs)
LR finder stopped early due to diverging loss.
Restored states from the checkpoint file at /content/lr_find_temp_model.ckpt
Learning rate set to 3.311311214825911e-05
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]

  | Name              | Type             | Params
-------------------------------------------------------
0 | feature_extractor | ResNet           | 25.6 M
1 | classifier        | Linear           | 5.0 K 
2 | loss_fn           | CrossEntropyLoss | 0     
-------------------------------------------------------
25.6 M    Trainable params
0         Non-trainable params
25.6 M    Total params
102.248   Total estimated model params size (MB)


1

In [ ]:
def evaluate_results(loader):
    model.eval()
    correct = 0
    total = 0
    
    with torch.no_grad():
        for x, y in iter(loader):        
            x = x.to(device)
            y = y.to(device)
            preds = model(x)
            _, predicted = torch.max(preds, 1)

            correct += (predicted == y).sum().item()
            total += len(y)
    return (correct / total)

In [ ]:
# Check the Validation Results
validation_loader = cassava_data.val_dataloader()
evaluate_results(validation_loader)